In [1]:
import pandas as pd
import numpy as np
import torch, time
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn import metrics
import os, csv

In [2]:
# --- Ruta del dataset ---
ruta_carpeta_actual = os.getcwd()
ruta_carpeta_raiz = os.path.dirname(ruta_carpeta_actual)
csv_path  = os.path.join(ruta_carpeta_raiz, "dataset", "PuntosMuestra_CR_2023.csv")

# --- Cargar CSV ---
df = pd.read_csv(csv_path)
df = df[df["CATEGORIA"] != 2].copy()
mapa = {old: new for new, old in enumerate(sorted(df["CATEGORIA"].unique()))}
df["CATEGORIA"] = df["CATEGORIA"].map(mapa)

y = df["CATEGORIA"].values
X = df.drop(columns=["CATEGORIA","lon","lat","year"]).values
X = StandardScaler().fit_transform(X)

# Modelo RF

In [3]:
from sklearn.ensemble import RandomForestClassifier
from utils_eval import evaluar_modelo

# --- RandomForest ---
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
resultados_rf = evaluar_modelo("RandomForest", rf, X, y)

# Guardar cada fold en CSV
with open(os.path.join(ruta_carpeta_actual, "resultados_rf.csv"), "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["fold", "OA", "F1_macro", "clase_dificil", "F1_clases", "matriz_confusion"])
    for r in resultados_rf:
        writer.writerow([r["fold"], r["OA"], r["F1_macro"], r["clase_dificil"], r["F1_clases"], r["matriz_confusion"].tolist()])

[RandomForest] Fold 1: OA=0.881, F1m=0.819, Clase más difícil=5
[RandomForest] Fold 2: OA=0.881, F1m=0.809, Clase más difícil=4
[RandomForest] Fold 3: OA=0.884, F1m=0.821, Clase más difícil=5
[RandomForest] Fold 4: OA=0.884, F1m=0.822, Clase más difícil=5
[RandomForest] Fold 5: OA=0.881, F1m=0.815, Clase más difícil=4


## Modelo MLP

In [4]:
import xgboost as xgb
from utils_eval import evaluar_modelo

# --- XGBoost ---
xgb_model = xgb.XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, n_jobs=-1, random_state=42)
evaluar_modelo("XGBoost", xgb_model, X, y)

# Guardar cada fold en CSV
with open(os.path.join(ruta_carpeta_actual, "resultados_xgb.csv"), "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["fold", "OA", "F1_macro", "clase_dificil", "F1_clases", "matriz_confusion"])
    for r in resultados_rf:
        writer.writerow([r["fold"], r["OA"], r["F1_macro"], r["clase_dificil"], r["F1_clases"], r["matriz_confusion"].tolist()])


[XGBoost] Fold 1: OA=0.888, F1m=0.832, Clase más difícil=5
[XGBoost] Fold 2: OA=0.889, F1m=0.825, Clase más difícil=4
[XGBoost] Fold 3: OA=0.892, F1m=0.832, Clase más difícil=5
[XGBoost] Fold 4: OA=0.884, F1m=0.823, Clase más difícil=5
[XGBoost] Fold 5: OA=0.888, F1m=0.829, Clase más difícil=5
